In [1]:
import pandas as pd
import requests
import numpy as np
import itertools
from random import shuffle
from tqdm import tqdm

In [2]:
demand_pub_oracle = pd.read_csv('./Data/Demand_LB_pred.csv', header=None)
solar_pub_oracle = pd.read_csv('./Data/Solar_LB_pred.csv', header=None)
price_pub_oracle = pd.read_csv('./Data/Price_LB_pred.csv', header=None)

In [3]:
bid_quantity = (demand_pub_oracle.values.flatten() - solar_pub_oracle.values.flatten()) * 1
bid_price = np.clip(price_pub_oracle.values.flatten() * 1, a_min=0, a_max=7)
submission = pd.DataFrame([bid_price, bid_quantity]).head()
submission.T.to_csv('23.csv', header=None, index=None)

In [4]:
def submit(name):
    url = 'http://interiit.tech:3000/upload'
    files = {'file': (name, open(name, 'rb'), '.text/csv', {'Content-Disposition': 'form-data'})}
    headers = {'Host': 'interiit.tech:3000', 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0',\
               'Accept': 'application/json, text/plain, */*', 'Accept-Language': 'en-US,en;q=0.5', 'Accept-Encoding': 'gzip, deflate',\
               'Referer': 'http://interiit.tech:3000/', 'Origin':'http://interiit.tech:3000'}
    r = requests.post(url, files=files, headers=headers)
    
def score(team_id):
    return pd.read_csv('http://interiit.tech:3000/results.csv').loc[team_id]['Score']

In [5]:
submit('23.csv')
score(23)

431776

# Grid search over Leaderboard

In [6]:
result = []
tples = list(itertools.product(np.arange(0.8, 1.25, 0.02), np.arange(0.8, 1.25, 0.02)))
shuffle(tples)
for alpha, beta in tples:
    bid_quantity = (demand_pub_oracle.values.flatten() - solar_pub_oracle.values.flatten()) * alpha
    bid_price = np.clip(price_pub_oracle.values.flatten() * beta, a_min=0, a_max=7)
    submission = pd.DataFrame([bid_price, bid_quantity]).head()
    submission.T.to_csv('23.csv', header=None, index=None)
    submit('23.csv')
    curr_score = score(23)
    result.append([alpha, beta, curr_score])

In [7]:
pd.DataFrame(result, columns=['Coefficient of Bid Quantity', 'Coefficient of Bid Price', 'Cost']).sort_values('Cost').reset_index(drop=True)

,Coefficient of Bid Quantity,Coefficient of Bid Price,Cost
0,1.0,1.2,324100
1,0.8,1.2,365583
2,1.2,1.2,372174
3,1.0,1.0,431776
4,1.2,1.0,441811
5,0.8,1.0,458421
6,0.8,0.8,576957
7,1.2,0.8,576957
8,1.0,0.8,576957


In [8]:
bid_quantity = (demand_pub_oracle.values.flatten() - solar_pub_oracle.values.flatten()) * 0.96
bid_price = np.clip(price_pub_oracle.values.flatten() * 1.18, a_min=0, a_max=7)
submission = pd.DataFrame([bid_price, bid_quantity]).head()
submission.T.to_csv('13.csv', header=None, index=None)
submit('13.csv')

In [9]:
result = pd.read_csv('http://interiit.tech:3000/results.csv', index_col='College Code').sort_values('Score')
result

,IIT Name,Score
College Code,,
15,IIT Guwahati,295632
11,IIT Kharagpur,309636
18,IIT Patna,312295
19,IIT Roorkee,314439
1,IIT Madras,316574
20,IIT Ropar,316819
7,IIT Bombay,318405
13,IIT Varanasi,319635
17,IIT Kanpur,328352


In [10]:
result['Score'] = (result['Score'].min() / result['Score']) * 20
result

,IIT Name,Score
College Code,,
15,IIT Guwahati,20.000000
11,IIT Kharagpur,19.095454
18,IIT Patna,18.932868
19,IIT Roorkee,18.803774
1,IIT Madras,18.676960
20,IIT Ropar,18.662517
7,IIT Bombay,18.569558
13,IIT Varanasi,18.498099
17,IIT Kanpur,18.007017
